In [113]:
import random
import networkx as nx
import itertools
import time

In [114]:
def is_resolving_set(B, G):
    B = set(B).intersection(G.nodes)  # filtriraj samo važeće čvorove
    distances = {}
    for v in G.nodes:
        distances[v] = [nx.shortest_path_length(G, v, u) for u in B]
    
    for v1, v2 in itertools.combinations(G.nodes, 2):
        if distances[v1] == distances[v2]:
            return False
    return True

In [115]:
def load_graph(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            
            node1 = int(parts[0]) 
            node2 = int(parts[1])  
            G.add_edge(node1, node2) 
    return G

In [116]:
def load_gcol(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            if parts[0] == 'e':
                node1 = int(parts[1]) 
                node2 = int(parts[2])  
                G.add_edge(node1, node2) 
    return G

In [117]:
def fitness(individual, G, shortest_paths):
    resolving_set = {i for i, bit in enumerate(individual) if bit == 1}

    if not resolving_set:
        return float("inf")

    unresolved_pairs = 0
    for u, v in itertools.combinations(G.nodes, 2):
        if not any(abs(shortest_paths[u][j] - shortest_paths[v][j]) > 0 for j in resolving_set if j in shortest_paths[u]):
            unresolved_pairs += 1 

    return len(resolving_set) + 100 * unresolved_pairs

In [118]:
def tournament_selection(population, G, k=3):
    selected = random.sample(population, k)
    return min(selected, key=lambda ind: fitness(ind, G))

In [119]:
def weighted_selection(population, G,shortest_paths):
    fitness_values = [1 / (1 + fitness(ind, G,shortest_paths)) for ind in population]
    return random.choices(population, weights=fitness_values, k=2) 

In [120]:
def crossover(parent1, parent2,G ):
    point = random.randint(1, len(G.nodes) - 1)
    return parent1[:point] + parent2[point:], parent2[:point] + parent1[point:]

In [121]:
def crossover2(parent1, parent2, G):

    point1 = random.randint(1, len(G.nodes) - 1)
    point2 = random.randint(1, len(G.nodes) - 1)

    if point1 > point2:
        point1, point2 = point2, point1

    child1 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
    child2 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]
    
    return child1, child2

In [122]:
def mutate(individual, mutation_rate):
    new_individual = individual[:]
    for i in range(len(new_individual)):
        if new_individual[i] == 1 and random.random() < mutation_rate:
            new_individual[i] = 0 
    return new_individual

In [123]:
def greedy_population(graph):

    nodes = list(graph.nodes())
    node_index = {node: i for i, node in enumerate(nodes)}  # mapiranje čvorova na indekse

    degrees = dict(graph.degree())

    sorted_nodes = sorted(nodes, key=lambda x: degrees[x], reverse=True)

    def is_resolving_set(S):
        coords = {node: tuple(nx.shortest_path_length(graph, node, target) for target in S) for node in nodes}
        return len(set(coords.values())) == len(nodes)

    S = []
    for node in sorted_nodes:
        S.append(node)
        if is_resolving_set(S):
            break


    binary_vec = [1 if node in S else 0 for node in nodes]
    return binary_vec

In [124]:
def generate_population(size,G):
    return [[random.randint(0, 1) for _ in range(len(G.nodes))] for _ in range(size)]

In [125]:
def generate_population_greedy(size,G):
    return  [greedy_population(G) for _ in range(size)]

In [126]:
def genetic_algorithm(G, shortest_paths,stop,generations, pop_size, crossover_rate, mutation_rate):
    population = generate_population(pop_size, G)

    global_best_solution = None
    global_best_fitness = float('inf')

    for gen in range(generations):
        new_population = []

        for _ in range(pop_size // 2):
            parent1, parent2 = weighted_selection(population, G, shortest_paths)

            # Ukrštanje
            if random.random() < crossover_rate:
                child1, child2 = crossover2(parent1, parent2, G)
            else:
                child1, child2 = parent1[:], parent2[:]

            # Mutacija
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            new_population.extend([child1, child2])

        population = new_population

        # Pronalazak najbolje jedinke u ovoj generaciji
        best_solution = min(population, key=lambda ind: fitness(ind, G, shortest_paths))
        best_fitness = fitness(best_solution, G, shortest_paths)

        print(f"Generacija {gen + 1}, Najbolji fitnes: {best_fitness}")

        # Ažuriranje globalno najboljeg rješenja
        if best_fitness < global_best_fitness:
            global_best_fitness = best_fitness
            global_best_solution = best_solution[:]
            no_improvement = 0
        else:
            no_improvement += 1

        if no_improvement >= stop:
            print(f"Prekid: nema poboljšanja tokom {stop} generacija.")
            break

    optimal_set = {i for i, bit in enumerate(global_best_solution) if bit == 1}
    print("\nOptimalni rješavajući skup:", optimal_set, len(optimal_set))
    return optimal_set,len(optimal_set)

In [127]:
G = load_gcol('grafovi\gcol\gcol30.txt')
if not nx.is_connected(G):
    print("Graf nije povezan!")
    largest_cc = max(nx.connected_components(G), key=len)
    G = G.subgraph(largest_cc).copy()
shortest_paths = dict(nx.all_pairs_shortest_path_length(G))

<>:1: SyntaxWarning: invalid escape sequence '\g'
<>:1: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Korisnik\AppData\Local\Temp\ipykernel_465112\3821854303.py:1: SyntaxWarning: invalid escape sequence '\g'
  G = load_gcol('grafovi\gcol\gcol30.txt')


In [128]:
param_grid = {
    'stop':[3,5],
    'pop_size': [20,30],
    'mutation_rate': [0.2],
    'crossover_rate': [0.6],
    'generations': [50]
}

best_score = float('inf')
best_params = None

for combo in itertools.product(*param_grid.values()):
    params = dict(zip(param_grid.keys(), combo))
    
    print(f"Testing: {params}")
    
    start_time = time.time()  # ⏱️ početak
    os,score = genetic_algorithm(G,shortest_paths,**params)
    end_time = time.time()    # ⏱️ kraj
    
    duration = end_time - start_time
    print(f"🔁 Rezultat: {score} | ⏱️ Vrijeme: {duration:.2f} sekundi")
    
    if (score < best_score) or (score == best_score and duration < duration1):
        if(is_resolving_set(os,G)):
            best_score = score
            best_params = params
            duration1 = duration

print("\n✅ Najbolji rezultat:", best_score)
print("📌 Najbolji parametri:", best_params)
print("📌 Vrijeme:", duration1)

Testing: {'stop': 3, 'pop_size': 20, 'mutation_rate': 0.2, 'crossover_rate': 0.6, 'generations': 50}
Generacija 1, Najbolji fitnes: 105
Generacija 2, Najbolji fitnes: 79
Generacija 3, Najbolji fitnes: 61
Generacija 4, Najbolji fitnes: 47
Generacija 5, Najbolji fitnes: 38
Generacija 6, Najbolji fitnes: 30
Generacija 7, Najbolji fitnes: 23
Generacija 8, Najbolji fitnes: 18
Generacija 9, Najbolji fitnes: 14
Generacija 10, Najbolji fitnes: 14
Generacija 11, Najbolji fitnes: 14
Generacija 12, Najbolji fitnes: 17
Prekid: nema poboljšanja tokom 3 generacija.

Optimalni rješavajući skup: {259, 102, 7, 41, 171, 206, 81, 49, 21, 152, 185, 283, 124, 191} 14
🔁 Rezultat: 14 | ⏱️ Vrijeme: 52.44 sekundi
Testing: {'stop': 3, 'pop_size': 30, 'mutation_rate': 0.2, 'crossover_rate': 0.6, 'generations': 50}
Generacija 1, Najbolji fitnes: 101
Generacija 2, Najbolji fitnes: 80
Generacija 3, Najbolji fitnes: 64
Generacija 4, Najbolji fitnes: 37
Generacija 5, Najbolji fitnes: 30
Generacija 6, Najbolji fitnes: